# NLP Project Pt. 6: Building a Recommendation System

In [93]:
import pandas as pd
import numpy as np
pd.set_option('max_colwidth',200)
doc_topic_df = pd.read_pickle('NMF_doc_topic_df3.pkl')
doc_topic_df.head()

,0,1,2,3,4,5
0,0.047,0.064,0.011,0.023,0.088,0.092
1,0.044,0.022,0.012,0.259,0.000,0.013
2,0.131,0.018,0.000,0.000,0.001,0.026
3,0.113,0.025,0.039,0.000,0.013,0.020
4,0.011,0.010,0.000,0.000,0.017,0.150


In [94]:
corpus_df = pd.read_pickle('corpus_df_style3.pkl')
corpus_df.reset_index(inplace=True, drop=True)
corpus_df.head()

,URL,TITLE,AUTHOR,DATE,TEXT,WORDCOUNT,TEXT_ENTS,POS_COUNT,NOUN_SCORE,VERB_SCORE,ADJ_SCORE,ADV_SCORE,AVG_SENT_LENGTH,LEX_RICH,COMBO_GROUP
0,https://www.newyorker.com/magazine/2022/02/14/annunciation,ANNUNCIATION,LAUREN GROFF,"February 7, 2022","['Some nights, in my dreams, I find myself running through those hills above Palo Alto again. It is always just before dawn, and as I run I smell the sun-crisped fields, the sage, the eucalyptus. ...",9373,"[Palo Alto, Bay, Mountain View, New England, San Francisco, Chinatown, San Francisco, Redwood City, Mountain View, Titania, Germany, Feuerzangenbowle, Redwood City, Mountain View, New York, Caribb...","[1705, 1451, 586, 614]",0.391414,0.333104,0.134527,0.140955,25.389920,0.232551,1
1,https://www.newyorker.com/magazine/2022/02/07/once-removed,ONCE REMOVED,ALEXANDER MACLEOD,"January 31, 2022","['She did not want to visit the old lady.', 'Amy studied the stroller, then the bags, then her boyfriend and the baby. She checked her phone: 11:26a.m. It was time to go. Ninety degrees, ninety-pe...",7778,"[Inverness County, Nova Scotia, Ontario, Toronto, Montreal, Cape Breton, Turkey, Niagara Falls, Montreal]","[1231, 1266, 455, 682]",0.338745,0.348376,0.125206,0.187672,15.590643,0.223521,3
2,https://www.newyorker.com/magazine/2022/01/31/long-distance,LONG DISTANCE,AYSEGUL SAVAS,"January 24, 2022","['Lea changed the sheets when she got up. She’d bought flowers the previous day, tulips that she’d put on the dresser. There were carnations on the kitchen table, in a squat glass vase. She though...",4866,"[California, Rome, Rome, Rome, California, Rome, Trastevere, the Ponte Sublicio, Everest, Rome, Rome, Rome, California, Ostiense, Rome, San Pietro, Vincoli, Rome, Rome, London, California]","[763, 903, 284, 328]",0.334943,0.396400,0.124671,0.143986,15.984227,0.273125,3
3,https://www.newyorker.com/magazine/2022/01/24/whats-the-deal-hummingbird,"WHAT'S THE DEAL, HUMMINGBIRD?",ARTHUR KRYSTAL,"January 17, 2022","['On or around May 5th of 2020, he just stopped. He stopped exercising, stopped walking, stopped reading, stopped planning. He ate, drank, washed, and paid the bills, but that was it. He was seven...",3469,"[Prospect Park, Kentucky, Tanglewood, East, Provence, Montpellier, Nice, Brooklyn, West Orange, New Jersey, New Orleans, San Francisco, New York, Corfu, America, America, America, America, America...","[609, 603, 204, 187]",0.379913,0.376170,0.127261,0.116656,17.959799,0.379558,0
4,https://www.newyorker.com/magazine/2022/01/17/fireworks,FIREWORKS,GRAHAM SWIFT,"January 10, 2022","['It was late October, 1962. Russian missiles were being shipped to Cuba. Kennedy was having words with Khrushchev. The world might be coming to an end.', 'It was a common remark: “Cheer up, it’s ...",2687,"[Cuba, Harpers]","[407, 424, 147, 202]",0.344915,0.359322,0.124576,0.171186,15.189189,0.311994,3


In [95]:
def NMF_cosine_similarity_recommender(df1, story_ix, df2, k):
    from sklearn.metrics.pairwise import cosine_similarity
    
    #generate compatibility score for each story pair
    match_holder = []
    story1 = df1.loc[story_ix].to_numpy()
    for i in range(len(df1)):
        if i != story_ix:
            story2 = df1.loc[i].to_numpy()
            story_compatibility = cosine_similarity(story1.reshape(1,-1), story2.reshape(1, -1))
            match_holder.append([i, story_compatibility])
    
    #select the 5 highest scores for each story
    matches = []
    stories = []
    
    for value in match_holder:
        story_num = value[0]
        score = value[1][0][0]
        matches.append(score)
        stories.append(story_num)

    matches_array = np.array(matches)
        
    part = np.argpartition(matches, -k)[-k:]
    indices = part[np.argsort((-matches_array)[part])]
    for ii in indices:
        print(df2.loc[story_ix].TITLE, df2.loc[stories[ii]].TITLE, df2.loc[stories[ii]].AUTHOR, matches_array[ii])
        

NMF_cosine_similarity_recommender(doc_topic_df, story_ix=0, df2=corpus_df, k=5)

ANNUNCIATION MARRIAGE QUARANTINE KATE WALBERT 0.9594571629846803
ANNUNCIATION THE VALETUDINARIAN JOSHUA FERRIS 0.9475549987101808
ANNUNCIATION WES AMERIGO'S GIANT FEAR DAVID SCHICKLER 0.9393300906371058
ANNUNCIATION BALLOONS THOMAS MCGUANE 0.9371954655297052
ANNUNCIATION THE MIDNIGHT ZONE LAUREN GROFF 0.9212865081875881


In [121]:
def recommender(doc_topic_df, story_ix, story_df, num_recs, topicweight, styleweight, authorweight):
    from sklearn.metrics.pairwise import cosine_similarity
    
    recs = pd.DataFrame(columns=['TITLE','AUTHOR','URL',
                                 'TOPIC_SIMILARITY','STYLE_MATCH','AUTHOR_MATCH','REC_SCORE'])
    
    ## part 1: topic similarity
    #generate compatibility score for each story pair
    topic_match_holder = []
    story1 = doc_topic_df.loc[story_ix].to_numpy()
    for i in range(len(doc_topic_df)):
        if i != story_ix:
            story2 = doc_topic_df.loc[i].to_numpy()
            story_compatibility = cosine_similarity(story1.reshape(1,-1), story2.reshape(1, -1))
            topic_match_holder.append([i, story_compatibility])
    
    topic_matches = []
    topic_stories = []
    
    for tvalue in topic_match_holder:
        tstory_num = tvalue[0]
        tscore = tvalue[1][0][0]
        topic_matches.append(tscore)
        topic_stories.append(tstory_num)

    matches = np.array(topic_matches)
        
    part = np.argpartition(matches, -800)[-800:]
    indices = part[np.argsort((-matches)[part])]
    for ii in indices:
        if ii != story_ix:
            if (story_df.loc[ii].COMBO_GROUP == story_df.loc[story_ix].COMBO_GROUP) & \
            (story_df.loc[ii].AUTHOR == story_df.loc[story_ix].AUTHOR):
                recs.loc[ii] = [story_df.loc[ii].TITLE, story_df.loc[ii].AUTHOR, story_df.loc[ii].URL,
                            matches[ii], 1, 1, 0]
            elif (story_df.loc[ii].COMBO_GROUP == story_df.loc[story_ix].COMBO_GROUP):
                recs.loc[ii] = [story_df.loc[ii].TITLE, story_df.loc[ii].AUTHOR, story_df.loc[ii].URL,
                            matches[ii], 1, 0, 0]
            elif (story_df.loc[ii].AUTHOR == story_df.loc[story_ix].AUTHOR):
                recs.loc[ii] = [story_df.loc[ii].TITLE, story_df.loc[ii].AUTHOR, story_df.loc[ii].URL,
                            matches[ii], 0, 1, 0]
            else:
                recs.loc[ii] = [story_df.loc[ii].TITLE, story_df.loc[ii].AUTHOR, story_df.loc[ii].URL,
                            matches[ii], 0, 0, 0]
            
            
    #putting it all together
    recs['REC_SCORE'] = recs['TOPIC_SIMILARITY']*topicweight + recs['STYLE_MATCH']*styleweight + recs['AUTHOR_MATCH']*authorweight
    print(story_ix, story_df.loc[story_ix].TITLE, story_df.loc[story_ix].AUTHOR)
    return recs.sort_values(by=['REC_SCORE'], ascending=False)[0:num_recs]

In [122]:
annunciation_recs = recommender(doc_topic_df, story_ix=0, story_df=corpus_df, num_recs=10, topicweight=1, styleweight=1, authorweight=2)
annunciation_recs

0 ANNUNCIATION LAUREN GROFF


,TITLE,AUTHOR,URL,TOPIC_SIMILARITY,STYLE_MATCH,AUTHOR_MATCH,REC_SCORE
253,FLOWER HUNTERS,LAUREN GROFF,https://www.newyorker.com/magazine/2016/11/21/flower-hunters,0.779337,0,1,2.779337
216,DOGS GO WOLF,LAUREN GROFF,https://www.newyorker.com/magazine/2017/08/28/dogs-go-wolf,0.571060,0,1,2.57106
320,GHOSTS AND EMPTIES,LAUREN GROFF,https://www.newyorker.com/magazine/2015/07/20/ghosts-and-empties,0.562175,0,1,2.562175
279,THE MIDNIGHT ZONE,LAUREN GROFF,https://www.newyorker.com/magazine/2016/05/23/the-midnight-zone-by-lauren-groff,0.551774,0,1,2.551774
172,UNDER THE WAVE,LAUREN GROFF,https://www.newyorker.com/magazine/2018/07/09/under-the-wave,0.533075,0,1,2.533075
134,BRAWLER,LAUREN GROFF,https://www.newyorker.com/magazine/2019/05/13/brawler,0.519944,0,1,2.519944
51,THE WIND,LAUREN GROFF,https://www.newyorker.com/magazine/2021/02/01/the-wind,0.501993,0,1,2.501993
604,WAR DANCES,SHERMAN ALEXIE,https://www.newyorker.com/magazine/2009/08/10/war-dances,0.947555,1,0,1.947555
319,SILK BROCADE,TESSA HADLEY,https://www.newyorker.com/magazine/2015/07/27/silk-brocade,0.917587,1,0,1.917587
907,GALLATIN CANYON,THOMAS MCGUANE,https://www.newyorker.com/magazine/2003/01/13/gallatin-canyon,0.902726,1,0,1.902726


In [127]:
recs = recommender(doc_topic_df, story_ix=22, story_df=corpus_df, num_recs=5, topicweight=1, styleweight=0.5, authorweight=0)
recs

22 THE MOM OF BOLD ACTION GEORGE SAUNDERS


,TITLE,AUTHOR,URL,TOPIC_SIMILARITY,STYLE_MATCH,AUTHOR_MATCH,REC_SCORE
330,THE FREEZER CHEST,DORTHE NORS,https://www.newyorker.com/magazine/2015/05/25/the-freezer-chest,0.969860,1,0,1.46986
749,THE PHOTOGRAPH,RODDY DOYLE,https://www.newyorker.com/magazine/2006/10/16/the-photograph-3,0.960192,1,0,1.460192
717,FAITH,WILLIAM TREVOR,https://www.newyorker.com/magazine/2007/06/04/faith-5,0.933542,1,0,1.433542
562,CORRIE,ALICE MUNRO,https://www.newyorker.com/magazine/2010/10/11/corrie,0.925821,1,0,1.425821
715,ROY SPIVEY,MIRANDA JULY,https://www.newyorker.com/magazine/2007/06/11/roy-spivey,0.909554,1,0,1.409554


In [128]:
recommender(doc_topic_df, story_ix=30, story_df=corpus_df, num_recs=5, topicweight=1, styleweight=1, authorweight=1)

30 UNREAD MESSAGES SALLY ROONEY


,TITLE,AUTHOR,URL,TOPIC_SIMILARITY,STYLE_MATCH,AUTHOR_MATCH,REC_SCORE
917,OF MYSTERY THERE IS NO END,LEONARD MICHAELS,https://www.newyorker.com/magazine/2002/04/08/of-mystery-there-is-no-end,0.998907,1,0,1.998907
244,QUARANTINE,ALIX OHLIN,https://www.newyorker.com/magazine/2017/01/30/quarantine-by-alex-ohlin,0.998128,1,0,1.998128
276,MAYBE IT WAS THE DISTANCE,JONATHAN SAFRAN FOER,https://www.newyorker.com/magazine/2016/06/06/maybe-it-was-the-distance-by-jonathan-safran-foer,0.997703,1,0,1.997703
286,GOD'S WORK,KEVIN CANTY,https://www.newyorker.com/magazine/2016/04/04/gods-work,0.997703,1,0,1.997703
691,THE VISITOR,MARISA SILVER,https://www.newyorker.com/magazine/2007/12/03/the-visitor-7,0.997703,1,0,1.997703


In [106]:
corpus_df[corpus_df.AUTHOR == 'SALLY ROONEY']

,URL,TITLE,AUTHOR,DATE,TEXT,WORDCOUNT,TEXT_ENTS,POS_COUNT,NOUN_SCORE,VERB_SCORE,ADJ_SCORE,ADV_SCORE,AVG_SENT_LENGTH,LEX_RICH,COMBO_GROUP
30,https://www.newyorker.com/magazine/2021/07/12/unread-messages,UNREAD MESSAGES,SALLY ROONEY,"July 5, 2021","['At twenty past twelve on a Wednesday afternoon, a woman sat behind a desk in a shared office in Dublin city center, scrolling through a text document. She had very dark hair, swept back loosely ...",9027,"[Dublin, County Galway, Oxford, England, England, Dublin, Dublin, Paris, St. Augustine, Paris, Paris, Dublin, Dublin, New York, Dublin, Dublin, London, London, Britain]","[1656, 1625, 525, 540]",0.381040,0.373907,0.120801,0.124252,14.452418,0.220785,2
142,https://www.newyorker.com/magazine/2019/03/18/color-and-light,COLOR AND LIGHT,SALLY ROONEY,"March 11, 2019","['The first time he sees her she’s getting into his brother’s car. He’s sitting in the back seat and she gets into the front, closing the passenger door behind her. Then she notices him. She crane...",6291,"[Paris, Aidan]","[927, 1182, 342, 523]",0.311701,0.397445,0.114997,0.175857,14.691275,0.211914,3
